Užitecne funkce Pythonu

In [4]:
my_string = "to be Or NOT to be"
my_list = my_string.split()
print 'Delka stringu je:', len(my_string)
print 'String obsahuje slova:', my_string.split()
print 'String obsahuje unikatni slova:', set(my_string.split())
print 'String prevedeny na mala pismena: ', my_string.lower()
print 'Delka pole (pocet slov) je:', len(my_list)
print 'Prvni prvek v poli je:', my_list[0]
print 'Posledni prvek v poli je:', my_list[-1]

Delka stringu je: 18
String obsahuje slova: ['to', 'be', 'Or', 'NOT', 'to', 'be']
String obsahuje unikatni slova: {'be', 'Or', 'NOT', 'to'}
String prevedeny na mala pismena:  to be or not to be
Delka pole (pocet slov) je: 6
Prvni prvek v poli je: to
Posledni prvek v poli je: be


## Spustit interaktivni shell Spark-pythonu

`pyspark --master yarn --num-executors 4`

### Priklad z prednasky: word count (nebo Hello hadoop)

In [ ]:
def split_string(verse):    
    return verse.split(' ')

lines = sc.textFile("/user/pascepet/data/bible.txt")
words = lines.flatMap(split_string)
pairs = words.map(lambda word: (word, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
counts.take(10)

## Ukol c. 1

### Zadani

Upravte priklad z prednasky o nasledujici vylepseni (v libovolnem poradi):

1. Vyberte RDD, ktere je vhodne si pro dalsi vypocty kesovat, a nakesujte ho.
1. Na konci seradte slova podle cetnosti sestupne (Hint: sortBy nebo sortByKey).
1. Pocitejte slovo jako stejne bez ohledu na velikost pismen.
1. Neberte v uvahu zacatek radku (nazev biblicke knihy a kod kapitola:vers -- je oddelen od textu tabulatorem).
1. Odstrante z textu vsechny nealfanumericke znaky (napr. '.', ':', '-', -- nebo aspon nektere z nich)

### Data

`/user/pascepet/data/bible.txt` na HDFS

**1. kesovani:** napr.  
`words.cache()` nebo `words = lines.flatMap(lambda line: line.split(" ")).cache()`

**2. serazeni slov podle cetnosti sestupne:**  
`counts_sorted = counts.sortBy(lambda x: x[1], ascending=False)`

**3. slova stejna bez ohledu na velikost pismen -- napr. prevodem slov na mala pismena:**  
`words = words.map(lambda word: word.lower())`

**4. oddeleni identifikace radku:**  
`lines = lines.map(lambda line: line.split("\t")[1])`

**5. odstraneni nealfanumerickych znaku:**  
* vyhozeni urciteho nealfanum. znaku:  
`words = words.map(lambda word: word.replace('.', '')`
* vyhozeni vsech nealfanum. znaku:  
`import re`  
`words = words.map(lambda word: re.sub(r'\W+', '', word))`

## Ukol c. 2

### Zadani 

a) Spocitejte pocet slov v kazdem versi (1 vers = 1 radek) a najdete verse s nejvetsim a nejmensim poctem slov.  
b) Provedte stejny vypocet jako v bode a), ale pocitejte jen unikatni slova v kazdem versi.

### Data

`/user/pascepet/bible.txt` na HDFS

### Ocekavany vystup

| verse_id | pocet_slov |
|:---------|:-----------|

----

In [ ]:
# kod obsahuje reseni pro a) i b) zaroven
# nacteni RDD
lines = sc.textFile("/user/pascepet/data/bible.txt") 

# rozdeleni radku na id a text, nakesujeme pro zrychleni opakovaneho behu
lines2 = lines.map(lambda line: line.split("\t"))

# vytvorime dvojici (id radku, list se slovy), nakesujeme pro zrychleni opakovaneho behu
lines3 = lines2.map(lambda line: (line[0], line[1].split(" ")))
# bylo by mozne slova ocistit jako v ukolu c. 1, tj. prevest na mala pismena, vyhodit nealfanum. znaky apod.

# spocteme slova a unikatni slova
counts = lines3.map(lambda line: (line[0], len(line[1]))).cache()
counts_uniq = lines3.map(lambda line: (line[0], len(set(line[1])))).cache()

# setrideni radku podle poctu slov a dotaz na prvni prvek
counts_sorted_asc = counts.sortBy(lambda x: x[1], True)
counts_sorted_desc = counts.sortBy(lambda x: x[1], False)
counts_sorted_asc.take(1)
counts_sorted_desc.take(1)
counts_uniq_sorted_asc = counts_uniq.sortBy(lambda x: x[1], True)
counts_uniq_sorted_desc = counts_uniq.sortBy(lambda x: x[1], False)
counts_uniq_sorted_asc.take(1)
counts_uniq_sorted_desc.take(1)

## Ukol c. 3

### Zadani

a) Zjistete, ktery stat ma nejvyssi prumernou teplotu z mereni jen za mesice 6--8. Teplotu uvedte ve stupnich Celsia.  
b) Pro kazdy mesic vypiste stat, ktery ma nejvyssi prumernou teplotu z mereni za tento mesic.

### Data

1. Na lokalnim filesystemu (tj. ne na HDFS) je vychozi soubor `/home/pascepet/fel_bigdata/data/teplota-usa.zip`. Tento soubor si zkopírujte do svého pracovního adresáře na lokálním filesystemu a rozbalte si ho tam.
1. Po rozbaleni dostanete dva soubory CSV. Spojte si je do souboru `teplota.csv`.
1. Soubor `teplota.csv` zkopirujte na HDFS do sveho pracovniho adresare.

### Vstupni data

CSV soubor ma jako oddelovac znak ','. Take obsahuje hlavicky s nazvy sloupcu, ktere je potreba pri zpracovani odstranit.    
Teplota je uvedena v 10 * stupne Fahrenheita. Nektere radky neobsahuji zadnou namerenou teplotu (prazdny retezec).

**Sloupce:** id_stanice, mesic, den, hodina, teplota, flag, latitude, longitude, vyska, stat, nazev


### Ocekavany vystup

V zadani a)

| stat | prum_teplota | 
|:-----|:-------------|


V zadani b)

| mesic | stat | prum_teplota | 
|:------|:-----|:-------------|

----

In [ ]:
import re

# nacteni RDD, odfiltrovani nezadoucich radku
# !!! zde je treba dosadit cestu do vlastniho user adresare na HDFS
lines = sc.textFile("/user/pascepet/data/teplota.csv")
lines2 = lines.filter(lambda line: not(re.match(r'stanice', line)))

# prevod radku na strukturu ((mesic, stat), teplota)
recs = lines2.map(lambda line: line.split(","))
# vyhozeni prazdnych udaju o teplote
recs = recs.filter(lambda rec: rec[4]!='')
# konverze z Fahrenheita na Celsia
recs = recs.map(lambda rec: ((int(rec[1]), rec[9]), (float(rec[4])/10-32)*5/9))
# nakesovani
recs = recs.persist(StorageLevel.MEMORY_AND_DISK)

# uloha a) stat s nejvetsi prumernou teplotou za mesice 6-8
# vyhodit udaje za jine mesice
recs1 = recs.filter(lambda rec: rec[0][0] in range(6,9))
# ponecha jen nazev statu (mesic nas uz nezajima), k tomu pripravi dvojice pro agregaci
recs1 = recs1.map(lambda rec: (rec[0][1], (1, rec[1])))
result1 = recs1.reduceByKey(lambda s,t: (s[0]+t[0], s[1]+t[1]))  # agregace po statech - pocet a soucet 
result1 = result1.map(lambda res: (res[0], res[1][1]/res[1][0])) # prumer = soucet/pocet
result1 = result1.sortBy(lambda res: res[1], False)
result1.take(1)

# uloha b) za kazdy mesic stat s nejvetsi prumernou teplotou
# spocita se agregace za kazdou dvojici (mesic, stat)
recs2 = recs.map(lambda rec: (rec[0], (1, rec[1])))
result2 = recs2.reduceByKey(lambda s,t: (s[0]+t[0], s[1]+t[1]))
result2 = result2.map(lambda res: (res[0], res[1][1]/res[1][0])) # nyni mame za kazdou dvojici (mesic, stat) prumernou teplotu
result2 = result2.map(lambda res: (res[0][0], (res[0][1], res[1]))) # dame mesic jako klic a mezi staty budeme hledat maximum 
result3 = result2.reduceByKey(lambda s,t: (t if t[1]>s[1] else s))
result3 = result3.sortBy(lambda res: res[0], True)
result3.collect()

## Bonusovy ukol

### Zadani

V ukolu 1 vyradte ze zpracovani tzv. stop-slova.

Hint: Pouzijte [Spark broadcast](https://spark.apache.org/docs/latest/rdd-programming-guide.html#broadcast-variables)

### Data

`/user/pascepet/data/bible.txt` na HDFS    
`/user/pascepet/data/stopwords.txt` na HDFS

In [ ]:
# nacist stop slova do mnoziny muzete pomoci prikazu
with open('/user/pascepet/data/stopwords.txt') as stopwords_file:
    stopwords = set([x.replace('\n', '') for x in stopwords_file.readlines()])

Pripravuje se :-)